In [1]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import pathlib
import cv2
import argparse
import io
import os
import scipy.misc
import numpy as np
import time
import glob
from IPython.display import display
from six import BytesIO
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings


In [2]:
# # PROVIDE PATH TO MODEL DIRECTORY
# PATH_TO_MODEL_DIR = 'D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/exported-models/03-my_model'

# # PROVIDE PATH TO LABEL MAP
# PATH_TO_LABELS = 'D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/annotations/label_map.pbtxt'

# # PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
# MIN_CONF_THRESH = float(0.6)

# # LOAD THE MODEL

# import time
# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as viz_utils

# PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

# print('Loading model...', end='')
# start_time = time.time()

# # LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
# detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# end_time = time.time()
# elapsed_time = end_time - start_time
# print('Done! Took {} seconds'.format(elapsed_time))

# # LOAD LABEL MAP DATA FOR PLOTTING

# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
#                                                                     use_display_name=True)

### Convert Model to TFLITE

In [11]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/exported-models/03-my_model_tflite/saved_model') # path to the SavedModel directory
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
# tflite_model = converter.convert()

In [20]:
# Save the model.
with open(r'D:\Pemrograman\Python\Project\Hidup-Sehat-Machine-Learning\ObjectDetection\workspace\exported-models\03-my_model_tflite\model.tflite', 'wb') as f:
  f.write(tflite_quant_model)

### MetaData

In [1]:
# Task Library expects label files that are in the same format as the one below.

_MODEL_PATH = "D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/exported-models/03-my_model_tflite/model.tflite"
_LABEL_FILE = "D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/annotations/labels.txt"
_SAVE_TO_PATH = "model_metadata_v2.tflite"

In [2]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

ObjectDetectorWriter = object_detector.MetadataWriter


# Normalization parameters is required when reprocessing the image. It is
# optional if the image pixel values are in range of [0, 255] and the input
# tensor is quantized to uint8. See the introduction for normalization and
# quantization parameters below for more details.
# https://www.tensorflow.org/lite/models/convert/metadata#normalization_and_quantization_parameters)


In [3]:
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],
    [_LABEL_FILE])

# Verify the metadata generated by metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

{
  "name": "HidupSehat ObjectDetection v2",
  "description": "Identify 36 classes of foods The expected image is 640 x 640, with three channels (red, blue, and green) per pixel stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
      "output

In [60]:
# from tflite_support.metadata_writers import object_detector
# from tflite_support.metadata_writers import writer_utils
# from tflite_support import metadata
# import flatbuffers
# import os
# from tensorflow_lite_support.metadata import metadata_schema_py_generated as _metadata_fb
# from tensorflow_lite_support.metadata.python import metadata as _metadata
# from tensorflow_lite_support.metadata.python.metadata_writers import metadata_info
# from tensorflow_lite_support.metadata.python.metadata_writers import metadata_writer
# from tensorflow_lite_support.metadata.python.metadata_writers import writer_utils

# ObjectDetectorWriter = object_detector.MetadataWriter

# _MODEL_PATH = "D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/exported-models/03-my_model_tflite/model.tflite"
# _LABEL_FILE = "D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/annotations/labels.txt"
# _SAVE_TO_PATH = "D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/scripts/preprocessing/tes"




# model_meta = _metadata_fb.ModelMetadataT()
# model_meta.name = "HidupSehat Object Detection"
# model_meta.description = ("Identify 36 Classes of Foods")
# model_meta.version = 'v2'
# model_meta.author = "TensorFlow"
# model_meta.license = ("Apache License. Version 2.0 "
#                         "http://www.apache.org/licenses/LICENSE-2.0.")
# # Creates input info.
# input_meta = _metadata_fb.TensorMetadataT()
# input_meta.name = "image"
# input_meta.description = ("The expected image is 640 x 640, with three channels "
#     "(red, blue, and green) per pixel. Each value in the tensor is between"
#     " 0 and 255.")
# input_meta.content = _metadata_fb.ContentT()
# input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
# input_meta.content.contentProperties.colorSpace = (
#     _metadata_fb.ColorSpaceType.RGB)
# input_meta.content.contentPropertiesType = (
#     _metadata_fb.ContentProperties.ImageProperties)
# input_normalization = _metadata_fb.ProcessUnitT()
# input_normalization.optionsType = (
#     _metadata_fb.ProcessUnitOptions.NormalizationOptions)
# input_normalization.options = _metadata_fb.NormalizationOptionsT()
# input_normalization.options.mean = [127.5]
# input_normalization.options.std = [127.5]
# input_meta.processUnits = [input_normalization]
# input_stats = _metadata_fb.StatsT()
# input_stats.max = [255]
# input_stats.min = [0]
# input_meta.stats = input_stats

# # Creates outputs info.
# output_location_meta = _metadata_fb.TensorMetadataT()
# output_location_meta.name = "location"
# output_location_meta.description = "The locations of the detected boxes."
# output_location_meta.content = _metadata_fb.ContentT()
# output_location_meta.content.contentPropertiesType = (
#     _metadata_fb.ContentProperties.BoundingBoxProperties)
# output_location_meta.content.contentProperties = (
#     _metadata_fb.BoundingBoxPropertiesT())
# output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
# output_location_meta.content.contentProperties.type = (
#     _metadata_fb.BoundingBoxType.BOUNDARIES)
# output_location_meta.content.contentProperties.coordinateType = (
#     _metadata_fb.CoordinateType.RATIO)
# output_location_meta.content.range = _metadata_fb.ValueRangeT()
# output_location_meta.content.range.min = 2
# output_location_meta.content.range.max = 2

# output_class_meta = _metadata_fb.TensorMetadataT()
# output_class_meta.name = "category"
# output_class_meta.description = "The categories of the detected boxes."
# output_class_meta.content = _metadata_fb.ContentT()
# output_class_meta.content.contentPropertiesType = (
#     _metadata_fb.ContentProperties.FeatureProperties)
# output_class_meta.content.contentProperties = (
#     _metadata_fb.FeaturePropertiesT())
# output_class_meta.content.range = _metadata_fb.ValueRangeT()
# output_class_meta.content.range.min = 2
# output_class_meta.content.range.max = 2
# label_file = _metadata_fb.AssociatedFileT()
# label_file.name = os.path.basename(_LABEL_FILE)
# label_file.description = "Label of objects that this model can recognize."
# label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
# output_class_meta.associatedFiles = [label_file]

# output_score_meta = _metadata_fb.TensorMetadataT()
# output_score_meta.name = "score"
# output_score_meta.description = "The scores of the detected boxes."
# output_score_meta.content = _metadata_fb.ContentT()
# output_score_meta.content.contentPropertiesType = (
#     _metadata_fb.ContentProperties.FeatureProperties)
# output_score_meta.content.contentProperties = (
#     _metadata_fb.FeaturePropertiesT())
# output_score_meta.content.range = _metadata_fb.ValueRangeT()
# output_score_meta.content.range.min = 2
# output_score_meta.content.range.max = 2

# output_number_meta = _metadata_fb.TensorMetadataT()
# output_number_meta.name = "number of detections"
# output_number_meta.description = "The number of the detected boxes."
# output_number_meta.content = _metadata_fb.ContentT()
# output_number_meta.content.contentPropertiesType = (
#     _metadata_fb.ContentProperties.FeatureProperties)
# output_number_meta.content.contentProperties = (
#     _metadata_fb.FeaturePropertiesT())

# # Creates subgraph info.
# group = _metadata_fb.TensorGroupT()
# group.name = "detection result"
# group.tensorNames = [
#     output_location_meta.name, output_class_meta.name,
#     output_score_meta.name
# ]
# subgraph = _metadata_fb.SubGraphMetadataT()
# subgraph.inputTensorMetadata = [input_meta]
# subgraph.outputTensorMetadata = [
#     output_location_meta, output_class_meta, output_score_meta,
#     output_number_meta
# ]
# subgraph.outputTensorGroups = [group]
# model_meta.subgraphMetadata = [subgraph]

# b = flatbuffers.Builder(0)
# b.Finish(
#     model_meta.Pack(b),
#     _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
# metadata_buf = b.Output()

# model_basename = os.path.basename(_MODEL_PATH)
# model_basename
# export_model_path = os.path.join(_SAVE_TO_PATH, model_basename)
# tf.io.gfile.copy(_MODEL_PATH, export_model_path, overwrite=True)
# populator = _metadata.MetadataPopulator.with_model_file(export_model_path)
# populator.load_metadata_buffer(metadata_buf)

# populator.load_associated_files([_LABEL_FILE])

# populator.populate()

# displayer = _metadata.MetadataDisplayer.with_model_file(export_model_path)

# json_file = displayer.get_metadata_json()
# export_json_file = os.path.join(_SAVE_TO_PATH,
#                                   os.path.splitext(model_basename)[0] + ".json")
# with open(export_json_file, "w") as f:
#     f.write(json_file)

#### check meta data

In [5]:
from tflite_support import metadata as _metadata

displayer = _metadata.MetadataDisplayer.with_model_file(r'D:/Pemrograman/Python/Project/Hidup-Sehat-Machine-Learning/ObjectDetection/workspace/exported-models/03-my_model_tflite/model.tflite')
export_json_file = "extracted_metadata_model.json"
json_file = displayer.get_metadata_json()

# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
  f.write(json_file)

ValueError: The model does not have metadata.